In [1]:
# Initial imports

import os
import requests
import pandas as pd
import numpy as np
import datetime as dt
import alpaca_trade_api as tradeapi
import seaborn as sns
from dotenv import load_dotenv
from MCForecastTools import MCSimulation
from pathlib import Path

import matplotlib.pyplot as plt

from pandas_datareader import data as pdr

%matplotlib inline

# Reading the CSV files into DataFrames and perform any necessary data cleaning steps. After cleaning, combine all DataFrames into a single DataFrame.

Files (Data from Google Finance and time range from 3-1-2009 to 26-10-2022):
* BTC  
* ETH  
* BNB 
* XRP 
* ADA 
* SOL 

In [8]:
# Read in crypto price data 

# 3-1-2009 to 26-10-2022

start= pd.to_datetime('2009-01-03')
end=pd.to_datetime('2022-10-26')

ada_data = pdr.get_data_yahoo('ADA-USD', start, end)
btc_data = pdr.get_data_yahoo('BTC-USD', start, end)
eth_data = pdr.get_data_yahoo('ETH-USD', start, end)
sol_data = pdr.get_data_yahoo('SOL-USD', start, end)
xrp_data = pdr.get_data_yahoo('XRP-USD', start, end)
bnb_data = pdr.get_data_yahoo('BNB-USD', start, end)

sol_data
#sol_data['2020-04-10']

,High,Low,Open,Close,Volume,Adj Close
Date,,,,,,
2020-04-10,1.313487,0.694187,0.832005,0.951054,87364276,0.951054
2020-04-11,1.049073,0.765020,0.951054,0.776819,43862444,0.776819
2020-04-12,0.956670,0.762426,0.785448,0.882507,38736897,0.882507
2020-04-13,0.891603,0.773976,0.890760,0.777832,18211285,0.777832
2020-04-14,0.796472,0.628169,0.777832,0.661925,16747614,0.661925
...,...,...,...,...,...,...
2022-10-22,28.144390,27.557684,28.108328,28.013863,474234383,28.013863
2022-10-23,29.331539,27.857960,28.013676,29.270071,633157057,29.270071
2022-10-24,29.413946,28.215033,29.272520,28.310064,667625312,28.310064


In [9]:
#sol_data = sol_data['Close']

#sol_data.to_csv('my_data.csv')

sol_data = sol_data.drop(columns=['High','Low','Open','Volume','Adj Close'])

sol_data.to_csv('../Resources/SOL.csv')

# Crypto Selection
- According to market cap data from yahoo finance 
- Exclude cryptos closely linked to USD, e.g. USDT, USDC etc.
- Top 6 Cryptos which have the largest market cap

In [ ]:
ada_data = read_crypto_price('ADA')
btc_data = read_crypto_price('BTC')
eth_data = read_crypto_price('ETH')
sol_data = read_crypto_price('SOL')
xrp_data = read_crypto_price('XRP')
bnb_data = read_crypto_price('BNB')

# Data concatination, plotting and analysis

In [ ]:
# Combine all the price data into one dataframe
combined_price = pd.concat([btc_data,eth_data,bnb_data,xrp_data,ada_data,sol_data],axis='columns',join = 'inner')
combined_price

In [ ]:
# plot the price history of the 6 cryptos
combined_price.plot(figsize=(20,10),title='Price History of the 6 Cryptos')

In [ ]:
# Calculate Daily Returns
combined_daily_return = combined_price.pct_change()

# Rename the column to 'XXX D_Return'
combined_daily_return = combined_daily_return.rename(
    columns=
    {
        'BTC Close':'BTC D_Return',
        'ETH Close':'ETH D_Return',
        'BNB Close':'BNB D_Return',
        'XRP Close':'XRP D_Return',
        'ADA Close':'ADA D_Return',
        'SOL Close':'SOL D_Return'
    }
)
combined_daily_return.dropna(inplace=True)
combined_daily_return

In [ ]:
# plot the daily return of the 6 cryptos
combined_daily_return.plot(figsize=(20,10),title='Daily Return of All Cryptos')

In [ ]:
# Calculate cumulative returns of the 6 cryptos
cumulative_return = (1 + combined_daily_return).cumprod()-1

cumulative_return

In [ ]:
cumulative_return.describe()

In [ ]:
# Plot cumulative return of all the cryptos
cumulative_return.plot(figsize=(20,10),title='Cumulative Return of All Cryptos')

In [ ]:
# Calculate the daily standard deviations

combined_std = combined_daily_return.std()
combined_std

In [ ]:
# Plot the Standard Deviation of the 6 Cryptos

combined_std.plot(kind = 'bar',title='Standard Deviation of the 6 Cryptos')

# Comparison between _Dollar-Cost Averaging_ and _Lump Sum Investment_ in 5 years time range

In [ ]:
def dca_lump_compare(coin_data):

    # Within 5 years time period, get the close price once every month, calculate the amount by 100/close_price    
    # Get the sum amount and calculate the value by the final (close price * sum_amount)

    # Extract the past 5 years' data from coin_data, time range starts from '2017-09-01' and end '2022-08-01' totally 60 months
    coin_data_slice = coin_data['2017-09-01':'2022-08-01']
    coin_data_slice

    # Group by year and month and get the first price of each month using first() function
    coin_inv_price=coin_data_slice.groupby([coin_data_slice.index.year, coin_data_slice.index.month]).first()
    coin_inv_price
    # Group by year and month and get 

    # Calculate the total amount acquried for the $100 monthly investment

    amount = 0
    total_amount = 0
    i = 0

    coin_inv_price.reset_index(inplace=True)
    coin_inv_price

    df = coin_inv_price.describe()
    count = int(df.iloc[0,0])

    
    while i<count:
        price = coin_inv_price.iloc[i,2]
        amount = round(100/price,4)
        total_amount = round((total_amount + amount),4)
        print(f'Year:{coin_inv_price.iloc[i,0]}, Month:{coin_inv_price.iloc[i,1]}, Pirce:{price}, Amount Acquired:{amount}, Amount Totally Acquired:{total_amount}')
        i += 1

    dca_value = total_amount * price
    print(f'DCA acquired cryptos over the 5-year period valued at:{dca_value} ')

    # investment over 5 years total at $100*60 = $6000
    dca_profit = dca_value - 6000
    print(f'DCA profit over the 5-year period :{dca_profit} ')

    # A Lump Sum Investment Return Calculation

    coin_daily_return = coin_data_slice.pct_change()
    coin_cumprod = (coin_daily_return+1).cumprod()-1
    coin_cumprod.dropna(inplace=True)
    coin_cumprod

    initail_amount = 6000

    cum_return_coin = coin_cumprod.loc['2022-08-01',:]

    cum_return_coin

    lump_sum_profit = initail_amount * cum_return_coin

    print(f'cum_return_coin:{cum_return_coin}')

    print(f'compound_return_coin:{lump_sum_profit}')

    # profit comparison between lump sum Vs. DCA

    print('*****************************************************************************\n')


    print(f'Initial investment of a lump sum of 6000 Vs. Monthly Investment of $100 over 5 years period Comparison\n')

    print(f'DCA profit: {dca_profit}')

    print(f'Lump Sum profit: {lump_sum_profit}')


    print('\n*****************************************************************************')
    
    compare_list = [dca_profit,lump_sum_profit]
    
    return compare_list



In [ ]:
compare_list_btc = dca_lump_compare(btc_data)
compare_list_ada = dca_lump_compare(ada_data)
compare_list_sol = dca_lump_compare(sol_data)
compare_list_xrp = dca_lump_compare(xrp_data)
compare_list_bnb = dca_lump_compare(bnb_data)
compare_list_eth = dca_lump_compare(eth_data)

compare_list = [compare_list_btc,compare_list_eth,compare_list_ada,compare_list_sol,compare_list_xrp,compare_list_bnb]  
                


compare_list